# Evaluación del modelo oficial DocVQA DONUT

En este notebook se evalúa el modelo Donut oficial entrenado sobre el dataset Document Visual Question Answering (DocVQA) sobre un set de datos definido. Este modelo permitirá la extracción de cualquier dato en cualquier documento mediante una pregunta que se realiza al documento. Se obtiene la precisión del modelo y el índice de confianza que determinará si un documento será procesado por el modelo o derivado al backoffice para la extracción manual de los datos.

El modelo se evalúa sobre unas imágenes determinadas. En esta demostración, de forma muy simpliicada, se han seleccionado 21 muestras de la clase email del dataset Tobacco-3482. Se decide extraer el campo que define el nombre del emisor del mensaje.

Hay que tener en cuenta que los documentos aportados por el cliente en formato pdf, ya habrán sido procesados y convertidos a imagen en la fase de clasificación de los documentos, antes de hacer la inferencia sobre el modelo DocVQA.

## Configuración del entorno

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 29.4 MB/s 
     |████████████████████████████████| 120 kB 73.6 MB/s 


In [ ]:
!pip install -q datasets sentencepiece

     |████████████████████████████████| 365 kB 24.0 MB/s 
     |████████████████████████████████| 1.3 MB 45.8 MB/s 
     |████████████████████████████████| 115 kB 75.5 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 


## Cargar imágenes

Ubicar las imágenes en una nueva carpeta llamada `dataset` (aportadas en la documentación).

In [ ]:
import glob

# Load files
file_paths = glob.glob("dataset/*")
file_paths.sort()

print(file_paths)

['dataset/01.jpg', 'dataset/02.jpg', 'dataset/03.jpg', 'dataset/04.jpg', 'dataset/05.jpg', 'dataset/06.jpg', 'dataset/07.jpg', 'dataset/08.jpg', 'dataset/09.jpg', 'dataset/10.jpg', 'dataset/11.jpg', 'dataset/12.jpg', 'dataset/13.jpg', 'dataset/14.jpg', 'dataset/15.jpg', 'dataset/16.jpg', 'dataset/17.jpg', 'dataset/18.jpg', 'dataset/19.jpg', 'dataset/20.jpg', 'dataset/21.jpg']


In [ ]:
from pathlib import Path

file_names = []
for file_path in file_paths:
  file_names.append(Path(file_path).name)

print(file_names)

['01.jpg', '02.jpg', '03.jpg', '04.jpg', '05.jpg', '06.jpg', '07.jpg', '08.jpg', '09.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg']


Se definen las etiquetas reales extraídas manualmente. Para ello se aporta el  archivo `labels.txt` con el nombre real del emisor de cada email. Habrá que subir este archivo al directorio raíz (aportado en la documentación).

In [ ]:
# Get labels
to_labels = []
with open("labels.txt") as file:
    for line in file:
        to_labels.append(line.rstrip())

print(to_labels)

['rahn, carolyn', 'blacout@webtv.net', 'payne, maura', 'matthews, ellen w.', 'byron nelson', "o'brien, j. brice", 'lewis, leslic', 'stanley, ronnie l.', 'buckner, janet w.', 'cheryl harry', 'cheek, shelby l.', 'lyons, mary m.', 'smith, jan fulton', 'fagan, mike g.', 'mason, marie n.', 'adams, tim', '/nelsonj', 'smith, jan fulton', 'lee, kristeen', 'milder, ally', 'gary glass']


Se crea un dataframe donde almacenaremos la información a evaluar.

In [ ]:
import pandas as pd

data = {'file_name': file_names, 'y_real': to_labels, 'confidence':"", 'y_pred':"", 'eval_total':"", 'eval_partial':""}
df = pd.DataFrame.from_dict(data)

display(df)

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,01.jpg,"rahn, carolyn",,,,
1,02.jpg,blacout@webtv.net,,,,
2,03.jpg,"payne, maura",,,,
3,04.jpg,"matthews, ellen w.",,,,
4,05.jpg,byron nelson,,,,
5,06.jpg,"o'brien, j. brice",,,,
6,07.jpg,"lewis, leslic",,,,
7,08.jpg,"stanley, ronnie l.",,,,
8,09.jpg,"buckner, janet w.",,,,
9,10.jpg,cheryl harry,,,,


## Cargar el model y processor

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

## Evaluación del modelo

Definimos la pregunta que queremos formular al modelo. En los emails, el nombre del emisor viene definido por la etiqueta From, así que la pregunta puede ser simplemente: From?

In [ ]:
question = "From?"

Se importa la función que calcula la confianza en las predicciones.

In [ ]:
def calculate_confidence(outputs, decoder_input_ids):
  gen_sequences = outputs.sequences[:, decoder_input_ids.to(device).shape[-1]:-1]
  probs = torch.stack(outputs.scores, dim=1).softmax(-1)
  gen_probs = torch.gather(probs, 2, gen_sequences[:, :, None]).squeeze(-1)
  unique_prob_per_sequence = gen_probs.prod(-1)

  return unique_prob_per_sequence.item()

Se evalúa el modelo con cada imagen cargada.

In [ ]:
import torch
import cv2
import re

for file_path in file_paths:
  image = cv2.imread(file_path)
  file_name = Path(file_path).name

  # We prepare the image for the model using DonutProcessor
  pixel_values = processor(image, return_tensors="pt").pixel_values

  task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
  prompt = task_prompt.replace("{user_input}", question)
  decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)

  outputs = model.generate(pixel_values.to(device),
                                decoder_input_ids=decoder_input_ids.to(device),
                                max_length=model.decoder.config.max_position_embeddings,
                                early_stopping=True,
                                pad_token_id=processor.tokenizer.pad_token_id,
                                eos_token_id=processor.tokenizer.eos_token_id,
                                use_cache=True,
                                num_beams=1,
                                bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                return_dict_in_generate=True,
                                output_scores=True)

  seq = processor.batch_decode(outputs.sequences)[0]
  seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token

  # confidance calculation
  confidence = calculate_confidence(outputs, decoder_input_ids)

  # We can convert the generated sequence to JSON ans extract the answer
  answer = processor.token2json(seq)["answer"]

  df.loc[df["file_name"] == file_name, 'y_pred'] = answer
  df.loc[df["file_name"] == file_name, 'confidence'] = confidence

In [ ]:
display(df)

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,01.jpg,"rahn, carolyn",0.992295,"rahn, carolyn",,
1,02.jpg,blacout@webtv.net,0.944454,blacout@webtv.net,,
2,03.jpg,"payne, maura",0.971057,"payne, maura",,
3,04.jpg,"matthews, ellen w.",0.987351,"matthews, ellen w.",,
4,05.jpg,byron nelson,0.999644,byron nelson,,
5,06.jpg,"o'brien, j. brice",0.826224,"o'brien, j. brice",,
6,07.jpg,"lewis, leslic",0.586025,"lewis, leslic",,
7,08.jpg,"stanley, ronnie l.",0.779956,"stanley, ronnie l.",,
8,09.jpg,"buckner, janet w.",0.945968,"buckner, janet w.",,
9,10.jpg,cheryl harry,0.999027,cheryl harry,,


Se calcula en los campos eval_total y eval_partial si existe concordancia total o parcial entre el nombre real y el predicho por el modelo. Se ignoran las diferencias entre mayúsculas y minúsculas. Por concordancia parcial se definen aquellas correlaciones de tres o más letras entre la clase real y la predicha o viceversa.

In [ ]:
df['eval_total'] = df.apply(lambda row: 1 if row.y_real ==  row.y_pred else 0, axis=1)
df['eval_partial'] = df.apply(lambda row: 1 if len(row.y_real)>2 and len(row.y_pred)>2 and (row.y_pred in row.y_real or row.y_real in row.y_pred) else 0, axis=1)

display(df)

# mean accuracy total match
mean_acc = round(df["eval_total"].mean()*100, 2)
print("Accuracy: " + str(mean_acc) + "%")

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,01.jpg,"rahn, carolyn",0.992295,"rahn, carolyn",1,1
1,02.jpg,blacout@webtv.net,0.944454,blacout@webtv.net,1,1
2,03.jpg,"payne, maura",0.971057,"payne, maura",1,1
3,04.jpg,"matthews, ellen w.",0.987351,"matthews, ellen w.",1,1
4,05.jpg,byron nelson,0.999644,byron nelson,1,1
5,06.jpg,"o'brien, j. brice",0.826224,"o'brien, j. brice",1,1
6,07.jpg,"lewis, leslic",0.586025,"lewis, leslic",1,1
7,08.jpg,"stanley, ronnie l.",0.779956,"stanley, ronnie l.",1,1
8,09.jpg,"buckner, janet w.",0.945968,"buckner, janet w.",1,1
9,10.jpg,cheryl harry,0.999027,cheryl harry,1,1


Accuracy: 95.24%


Con una precisión del 95.24%, no cumplimos con el requisito del enunciado de una precisión del 98%.

En este caso, ya que como uno de los KPIs del enunciado es que se permite una automatización >= 60%, se descartan de la validación del modelo aquellas muestras que tengan una confianza inferior a un cierto índice de confianza.

In [ ]:
# Define a confidence index to balance between % of filtered data and accuracy
confidence_index = .5

# Filter data
df_filter = df[df['confidence'] > confidence_index]
display(df_filter)

# mean accuracy of filtered data
mean_fil_acc = round(df_filter["eval_total"].mean()*100, 2)

# % of filtered data
num_df = df.shape[0]
num_df_filter = df_filter.shape[0]
per_filtered = round(num_df_filter/num_df*100, 2)

print("Accuracy of filtered data: " + str(mean_fil_acc) + "%")
print("% of filtered data: " + str(per_filtered) + "%")

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,01.jpg,"rahn, carolyn",0.992295,"rahn, carolyn",1,1
1,02.jpg,blacout@webtv.net,0.944454,blacout@webtv.net,1,1
2,03.jpg,"payne, maura",0.971057,"payne, maura",1,1
3,04.jpg,"matthews, ellen w.",0.987351,"matthews, ellen w.",1,1
4,05.jpg,byron nelson,0.999644,byron nelson,1,1
5,06.jpg,"o'brien, j. brice",0.826224,"o'brien, j. brice",1,1
6,07.jpg,"lewis, leslic",0.586025,"lewis, leslic",1,1
7,08.jpg,"stanley, ronnie l.",0.779956,"stanley, ronnie l.",1,1
8,09.jpg,"buckner, janet w.",0.945968,"buckner, janet w.",1,1
9,10.jpg,cheryl harry,0.999027,cheryl harry,1,1


Accuracy of filtered data: 100.0%
% of filtered data: 95.24%


Con un índice de confianza de 0.5 cumplimos con los KPIs del enunciado:

*   Precisión 100.00% (objetivo>=98%)
*   Automatización 95.24% (objetivo>=60%)

Hay que tener en cuenta de que se han utilizado sólo 21 imágenes para evaluar el modelo. Para una correcta evaluación del modelo, se debería emplear un dataset con muchas más muestras.

Sin embargo, los testeos que he realizado desde la demo del modelo en Hugging Face son muy prometedores:
https://huggingface.co/spaces/nielsr/donut-docvqa

Este mismo ejercicio se podría realizar para cualquier documento y dato a extraer.